# Overview

This Jupyter Notebook takes in data from a Google Sheet that contains stop change details and their associated high level categories and outputs a JSON file for each line to be used in the MyBus tool.

The output file is used by the MyBus tool's results page and contains the Stop-level changes that are displayed there.

Run all cells to output files in the format `#-changes.json`

As of 5/28/21, this should output data for 141 lines.

In [8]:
import pandas as pd
import numpy as np
DATA_INPUT_PATH = '../data/input/'
DATA_OUTPUT_PATH = '../data/stop-changes/'

# Stop Changes

Data was compiled from spreadsheets and slides provided by Service Planners.

As of 5/29/21, there are 4794 rows.

In [2]:
import pandas as pd
GOOGLE_SHEET_LINK = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQC-r5V2B0xw886UtXO5TU1YXPy_HzyBoQXQOjDWD_qwmBBlvn3fyVsa-BHu0FymPM8gk4O_UxMtJsK/pub?output=csv"
# stop_changes = pd.read_csv(DATA_INPUT_PATH + 'stop_changes - ALL.csv',
#     usecols={'line', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})
stop_changes = pd.read_csv(GOOGLE_SHEET_LINK,
    usecols={'line', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})

In [11]:
# Set empty stop_ids to 0, cast all to int and then string
stop_changes.stop_id = stop_changes.stop_id.fillna(0)
stop_changes.stop_id = stop_changes.stop_id.astype('int')
stop_changes.stop_id = stop_changes.stop_id.astype('string')

#stop_changes.count()
# 5/29/21 - 4974 stop changes rows

#stop_changes[stop_changes.stop_id == '0']
# 5/29/21 - 2 stops with no stop id (== 0)

stop_changes.head()

,line,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled
0,2.0,14805,False,False,False,True,False,False,False
1,2.0,6424,False,False,False,True,False,False,False
2,4.0,5972,False,False,False,True,False,False,False
3,4.0,1070,False,False,False,True,False,False,False
4,4.0,5978,False,False,False,True,False,False,False


# Data Cleanup

## Results 

There are only 2 entries left with no `stop_id`.  They are for Line 234, Borden / Noble, which doesn't exist in the GTFS.

These need to be set to 0 in order to cast the column as type `int`.  Then cast the column to type `string` for standard comparisons with other data.


### Load GTFS Data

In [9]:
lines = pd.read_csv(DATA_INPUT_PATH + 'routes.txt', 
    usecols={'route_id', 'route_short_name'},
    dtype={'route_id':'string', 'route_short_name':'string'})
stops = pd.read_csv(DATA_INPUT_PATH + 'stops.txt',
    usecols=['stop_id','stop_name','stop_lat','stop_lon'],
    dtype={'stop_id':'string','stop_name':'string','stop_lat':'float64','stop_lon':'float64'})
stop_times = pd.read_csv(DATA_INPUT_PATH + 'stop_times.txt',
    usecols=['trip_id','stop_id','stop_sequence','stop_headsign'],
    dtype={'trip_id':'string','stop_id':'string','stop_sequence':'int64','stop_headsign':'string'})

FileNotFoundError: [Errno 2] No such file or directory: '../data/input/stops.txt'

### Create List of All Lines used in MyBus

In [102]:
# Add route_short_name values for the 901 (Orange Line) and 910/950 (Silver Line)
lines.loc[lines["route_id"] == '910-13139', 'route_short_name'] = '910/950'
lines.loc[lines["route_id"] == '901-13139', 'route_short_name'] = '901'

# Line 16/17 is listed as only '16' in the GTFS data even though headsigns show 17.
# Add line 17 back in.
lines.loc[lines["route_id"] == '16-13139', 'route_short_name'] = '16/17'

# Add back in lines: 177, 244, 489, 788.
lines.loc[len(lines.index)] = ['177', '177'] # no stops
lines.loc[len(lines.index)] = ['244', '244'] # has stops
lines.loc[len(lines.index)] = ['489', '489'] # has stops
lines.loc[len(lines.index)] = ['788', '788'] # no stops

# Remove the entries for the Dodger Express and L Line (Gold) Shuttle
lines = lines.loc[~lines["route_id"].isin(['DSE-HG'])]
lines = lines.loc[~lines["route_id"].isin(['DSE-US'])]
lines = lines.loc[~lines["route_id"].isin(['854-13139'])]

# Separate out the sister lines.
lines_array = lines.loc[lines['route_short_name'].str.contains('/'), 'route_short_name'].values

for i, l in enumerate(lines_array):
    id = lines.loc[lines['route_short_name'] == l]['route_id'].values[0]
    slash = l.find('/')
    line1 = l[:slash]
    line2 = l[slash+1:]
    
    lines = lines.loc[~lines["route_id"].isin([id])]
    newlines = pd.DataFrame([[id, line1], [id, line2]], columns=['route_id', 'route_short_name'])
    lines = lines.append(newlines, ignore_index=True)

# cast route_short_name to int32 so that we can sort by their integer value
lines = lines.astype({'route_short_name': 'int32'}).sort_values('route_short_name')
#lines.head(10)

lines_array = lines.loc[:, 'route_short_name'].values
#print(str(len(lines_array)) + ' lines')
#print(lines_array)

### Join stops and lines in a simplified view

In [103]:
# lines_and_stops = pd.merge(stop_times, stops, how="left", on="stop_id")
# simple_lines_stops = lines_and_stops[['stop_id','stop_headsign','stop_name']].copy()
# simple_lines_stops['line'] = np.nan
# counter = 1

# for line in lines_array:
#     line_regex = '^' + str(line) + '\s'
#     simple_lines_stops.loc[simple_lines_stops['stop_headsign'].str.contains(line_regex), 'line'] = line
    
#     counter += 1
#     if counter % 10 == 0:
#         print(str(counter) + ' lines processed')

## Line 45/127

One set of stop changes is listed as being for Line 45/127.  It's unclear whether the data should be shown for one or both, especially since the two lines overlap.  The stops with changes should be compared to the entirety of the stop list for both lines to see which one it matches.

The value in the `lines` column for these stop changes was listed as "45, 127".  After the below analysis, the `lines` value will be designated as 45.  Two stop_ids listed in the list of changes were not found in the GTFS data - 13271 and 10439.

In [104]:
# list of all stops with changes for line 45/127
# changes_45_127 = stop_changes[stop_changes.line == '45, 127']
# changes_45_127 = changes_45_127[['stop_id']].copy()

# list of all stops for line 45
# 32,340 total
# 145 after dropping duplicate stop_ids
# stops_45 = simple_lines_stops[simple_lines_stops.line == 45].drop_duplicates('stop_id')
# stops_45.head()

# list of all stops for line 127
# stops_127 = simple_lines_stops[simple_lines_stops.line == 127].drop_duplicates('stop_id')
# stops_127.head()

# merged_45 = pd.merge(changes_45_127, stops_45, how='left', on='stop_id', indicator=True)

# merged_45[merged_45._merge != 'both']

# only 2 results left-only, thus the stop changes match line 45.
# the 2 stops that are not part of line 45's stops: 13271, 10439

# these stop_ids don't even exist at all in the GTFS
#simple_lines_stops[simple_lines_stops.stop_id == '13271']
#simple_lines_stops[simple_lines_stops.stop_id == '10439']

# example of a stop_id that does exist
#simple_lines_stops[simple_lines_stops.stop_id == '2323']

# merged_127 = pd.merge(changes_45_127, stops_127, how='left', on='stop_id', indicator=True)
# merged_127[merged_127._merge != 'both'].count()

# 57 results left-only, thus the stop changes do not match line 127.

## Line 51/52 - 127

One set of stop changes is listed as being for Line 51/52-127.  It's unclear whether what line(s) these changes should apply to.

Based on the analysis below, the stop changes data will be duplicated and applied to both lines 51 and 52 since it affects both lines' stops that overlap.

In [105]:
# changes_51_52_127 = stop_changes[stop_changes.line == '51-52, 127']
# changes_51_52_127 = changes_51_52_127[['stop_id', 'stop_name']].copy()

# stops_51 = simple_lines_stops[simple_lines_stops.line == 51].drop_duplicates('stop_id')

# merged_51 = pd.merge(changes_51_52_127, stops_51, how='left', on='stop_id', indicator=True)
# merged_51[merged_51._merge != 'both']

# stops_52 = simple_lines_stops[simple_lines_stops.line == 52].drop_duplicates('stop_id')

# merged_52 = pd.merge(changes_51_52_127, stops_52, how='left', on='stop_id', indicator=True)
# merged_52[merged_52._merge != 'both']

# only one of the stops with changes does not show up for line 52
# it has to be a stop for line 51 only

# Merge other direction
# merged_51_right = pd.merge(stops_51, changes_51_52_127, how='left', on='stop_id', indicator=True)
# merged_51_right[merged_51_right._merge != 'both']

# Merge other direction
# merged_52_right = pd.merge(stops_52, changes_51_52_127, how='left', on='stop_id', indicator=True)
# merged_52_right[merged_52_right._merge != 'both']

## Line 757

Line 757 did not have stops listed in the changes spreadsheet.  

Data provided:
* 757	All Stops Except Adams, Washington, and NB at King - Service Replaced (38)
* 757	Adams - Service Replaced (2)
* 757	Washington - Service Replaced (2)
* 757	NB at King - Service Replaced (1)

Text on signage for the 3 special stops indicates that they have been relocated in addition to their service being replaced.

Signage quantities mostly match what is in the GTFS data.  First sign should have 39 copies, not 38.

Data manually added to stop_changes spreadsheet:
* Service replaced - ALL Stops (44 rows)
* Stop Relocated - 3 stops (5 rows)
    * Western / Adams (both directions)
    * Western / Washington (both directions)
    * Western / Martin Luther King Jr (Northbound)

In [106]:
# stops_757 = simple_lines_stops[simple_lines_stops.line == 757]
# stops_757 = stops_757.drop_duplicates('stop_id')
## 44 unique stop IDs for line 757

# stops_757[stops_757.stop_name.str.contains('Adams')]
# Western / Adams - 19147, 19148

# stops_757[stops_757.stop_name.str.contains('Washington')]
# Western / Washington - 65300052, 65300070

# stops_757[stops_757.stop_name.str.contains('King')]
# Western / Martin Luther King Jr - 65300050, 11257
# But need Northbound ONLY

# Get coordinates
# lines_and_stops[lines_and_stops.stop_id.isin(['65300050', '11257'])].drop_duplicates(subset=['stop_headsign','stop_id'])

# After plotting coordinates, northbound is stop_id 65300050

# stops_757[~stops_757.stop_id.isin(['19147', '19148', '65300052', '65300070', '65300050'])]
# 44 - 5 = 39 stop_ids

## SFV Stop Consolidation - Lines 92, 94, 164, 165, 224, 234, 240

Data for these stops was only provided in the form of signage, on which the stop names were listed.

Stop IDs found for 25 stop change entries.

NO stop ID found in the GTFS for Line 234, stop Borden / Noble.

In [107]:
# line_92 = simple_lines_stops[simple_lines_stops.line == 92].drop_duplicates('stop_id')

# line_92[line_92.stop_name.str.contains('Bellevue / Douglas')]
# 381, 8967

# line_92[line_92.stop_name.str.contains('Glendale / Earl')]
# 2202, 10743

# line_94 = simple_lines_stops[simple_lines_stops.line == 94].drop_duplicates('stop_id')

# line_94[line_94.stop_name.str.contains('San Fernando / Thompson')]
# 5855, 14231

# line_164 = simple_lines_stops[simple_lines_stops.line == 164].drop_duplicates('stop_id')

# line_164[line_164.stop_name.str.contains('Victory / Evergreen')]
# 8147, 16427

# line_164[line_164.stop_name.str.contains('Victory / Fairview')]
# 8151, 16431

# line_164[line_164.stop_name.str.contains('Victory / Lincoln')]
# 8155, 16435

# line_224 = simple_lines_stops[simple_lines_stops.line == 224].drop_duplicates('stop_id')

# line_224[line_224.stop_name.str.contains('Lankershim / Huston')]
# 11308, 2821

# line_224[line_224.stop_name.str.contains('Lankershim / Collins')]
# 2815, 11303

# line_234 = simple_lines_stops[simple_lines_stops.line == 234].drop_duplicates('stop_id')

# line_234[line_234.stop_name.str.contains('Borden / Noble')]
# doesn't exist in GTFS data

# line_234[line_234.stop_name.str.contains('Borden / Beaver')]
# 453, 20101

# line_234[line_234.stop_name.str.contains('Sayre / Wheeler')]
# 14509, 20086

# line_234[line_234.stop_name.str.contains('Sayre / Kismet')]
# 6127, 140737

# line_240 = simple_lines_stops[simple_lines_stops.line == 240].drop_duplicates('stop_id')

# line_240[line_240.stop_name.str.contains('Ventura / Dixie Canyon')]
# 15358, 7011

## Lines 111, 207

Stop names provided for these two lines, but no stop IDs.

Line 111, stop Florence / Orizaba is called Florence / Western on the other side of the street.

The GTFS data lists 3 stops named Florence / Western.  After looking at the coordinates, stop id 1960 is the correct one that is opposite the stop at Florence / Orizaba.

In [108]:
# line_111 = simple_lines_stops[simple_lines_stops.line == 111].drop_duplicates('stop_id')

# line_111[line_111.stop_name.str.contains('Florence / Grand')]
# 7643, 15984

# line_111[line_111.stop_name.str.contains('Florence / Orizaba')]
# 10532

# line_111[line_111.stop_name.str.contains('Florence / Western')]
# Which stop_id?  7648, 1960, 15989

# Get coordinates
# lines_and_stops[lines_and_stops.stop_id.isin(['10532', '7648', '1960', '15989'])].drop_duplicates('stop_id')

# Florence / Orizaba (10532)
# 33.952526,-118.127613,red,square,"Orizaba"

# Florence / Western (7648)
# 33.974447,-118.309231,blue,circle,"Western 1"

# Florence / Western (1960) <-- Correct location
# 33.952424,-118.127992,blue,circle,"Western 2"

# Florence / Western (15989)
# 33.974701,-118.308704,blue,circle,"Western 3"

# line_207 = simple_lines_stops[simple_lines_stops.line == 207].drop_duplicates('stop_id')

# line_207[line_207.stop_name.str.contains('Western / 94')]
# 140987, 14135

# line_207[line_207.stop_name.str.contains('Western / La Mirada')]
# 3989, 142120

## Line 90 - Ternimal 28 / East Lot

This was listed twice in the source spreadsheet under different travel directions but only the second instance had a stop id listed.

In [109]:
# line_90 = simple_lines_stops[simple_lines_stops.line == 90].drop_duplicates('stop_id')

# line_90[line_90.stop_name.str.contains('Terminal')]
# 6535

# Results After Cleanup

Now there are 4,974 rows of stop changes data.  The only missing stop_ids are for Line 234, Borden / Noble, because it doesn't exist in the GTFS.

Questions that came up for further analysis:

* What stop signs list additional applicable categories?  Specifically, that the stop has been relocated?  This probably applies primarily to service_changed, service_replaced, or route_changed categories.
* Can stop_headsign be used to determine travel direction data for that line's stops?

### How many Stops are in stop_changes but not the GTFS data?

In [1]:
# How many stops in stop changes are not in the GTFS data?
stops_not_in_gtfs = stop_changes[~stop_changes.stop_id.isin(stops.stop_id)]
stops_not_in_gtfs = stops_not_in_gtfs[['line', 'stop_id']]

stops_not_in_gtfs = stops_not_in_gtfs.sort_values(['line', 'stop_id'])
print(stops_not_in_gtfs.count())

stops_not_in_gtfs.to_json(DATA_OUTPUT_PATH + 'stops-not-in-gtfs.json', orient='records')

# 73 - 2(stop_id=0) = 71 stops in stop_changes data that don't exist in GTFS
# sampling of stop_ids that don't exist in the GTFS
# ['672','7929','7906','12827','9851','4716','4963','13559','13271','10439']

NameError: name 'stop_changes' is not defined

# General Analysis of GTFS Data

## Duplicate `stop_id`

Not counting the 40 entries with no `stop_id`, there are 505 `stop_id`s that occur at least 2 times.

There are 55 `stop_id`s that occur at least 3 times.  `30015` (SYL/SF Metrolink Station) occurs 14 times.

## Duplicate `line` & `stop_id` combo

After removing for non-existent stop_ids, there are 4506 unique line + stop_id combinations.

* 4317 combos occur 1 time, 189 combos occur multiple times
* 179 combos occur 2 times
* 10 combos occur at least 3 times

Within the 10 combos that that have at least 3 duplicate entries with the same `line` and `stop_id`, the line 237, stop_id 25001 combo has entries with service_changed and service_replaced selected.

## Merge

Join datasets to show the stop change categories for the line-stop combos with duplicates.  The 189 line-stop combos matched with 395 rows from the overall dataset.

From this identify where each line-stop combo has different change categories applied.

This dataframe was exported to CSV and loaded into Google Sheets for easier analysis.
Of those 395 rows, 178 line-stop combos have different categories listed.

In [110]:
# Find duplicate stop_ids
#stop_changes['stop_id'].value_counts()
#stop_changes['stop_id'].value_counts().loc[lambda x : x>2]

# stop_id == 30015 (14 occurences)
# SYL/SF Metrolink Station, different lines, directions, and locations
#stop_changes.loc[stop_changes['stop_id'] == 30015]

In [111]:
# Find duplicate line + stop_id combos

# exclude non-existent stop_ids
# stop_changes_existing_stopids = stop_changes.loc[stop_changes['stop_id'] != 0]

# stop_changes_existing_stopids = stop_changes_existing_stopids.groupby(['line', 'stop_id']).size().reset_index(name="count")
# stop_changes_existing_stopids = stop_changes_existing_stopids.loc[stop_changes_existing_stopids['count'] > 1]

# Output file with the stops that have duplicate rows.
# filtered_combos = pd.merge(stop_changes_existing_stopids, stop_changes, how='inner', on=['line', 'stop_id']).sort_values(by=['line', 'stop_id'])
# filtered_combos.to_csv(DATA_OUTPUT_PATH + 'stop_changes_duplicates.csv')

# Output Stop Changes Data for MyBus

Output a file for each line.

Create a list of lines by dropping duplicate lines.  Loop through the array and output all stop changes for each line into separate files.


In [112]:
# Create a list of unique line numbers using the list displayed in MyBus
# Read lines.json, which is an output of mybus-data.ipynb
# As of 5/28/21 - should contain 141 lines
lines_mybus = pd.read_json(DATA_OUTPUT_PATH + 'lines.json')

arr_lines_mybus = lines_mybus.loc[:, 'route_short_name'].values
len(arr_lines_mybus)
# 141 lines

# This uses the stop_changes dataset, which doesn't give us line numbers for which there are no stop changes
# lines_in_stop_changes = stop_changes['line'].drop_duplicates().reset_index()
# lines_in_stop_changes = lines_in_stop_changes.loc[:, 'line'].values

# len(lines_in_stop_changes)
# 71 lines in stop changes

141

In [113]:
# Loop through line numbers and output a file for each line with all data for that line
# As of 5/28/21 should output files for 141 lines
i = 1

for line in arr_lines_mybus:
    subset = stop_changes[stop_changes.line == line]
    filename = DATA_OUTPUT_PATH + 'changes/' + str(line) + '-changes.json'
    
    subset.to_json(filename, orient='records')
    
    print(str(i) + ': Line ' + filename + ' created')
    i += 1

1: Line data-output/changes/2-changes.json created
2: Line data-output/changes/4-changes.json created
3: Line data-output/changes/10-changes.json created
4: Line data-output/changes/14-changes.json created
5: Line data-output/changes/16-changes.json created
6: Line data-output/changes/17-changes.json created
7: Line data-output/changes/18-changes.json created
8: Line data-output/changes/20-changes.json created
9: Line data-output/changes/28-changes.json created
10: Line data-output/changes/30-changes.json created
11: Line data-output/changes/33-changes.json created
12: Line data-output/changes/35-changes.json created
13: Line data-output/changes/37-changes.json created
14: Line data-output/changes/38-changes.json created
15: Line data-output/changes/40-changes.json created
16: Line data-output/changes/45-changes.json created
17: Line data-output/changes/48-changes.json created
18: Line data-output/changes/51-changes.json created
19: Line data-output/changes/52-changes.json created
20: 